In [9]:
import numpy as np

In [67]:
#function that writes the output into lammps format
def writelammps(filename, Lx, Ly, Lz, TIMESTEP, r, v): #r,v are numpy arrays
    #WRITELAMMPS Write data to lammps file
    with open(filename, 'a') as fp:
        ip = len(r)
        fp.write('ITEM: TIMESTEP\n')
        fp.write(f'{TIMESTEP}\n')
        fp.write('ITEM: NUMBER OF ATOMS\n')
        fp.write(f'{ip}\n') # Nr of atoms
        fp.write('ITEM: BOX BOUNDS pp pp pp\n')
        fp.write(f'{0.0} {Lx}\n') #box size, x
        fp.write(f'{0.0} {Ly}\n')
        fp.write(f'{0.0} {Lz}\n')
        fp.write('ITEM: ATOMS id type x y z vx vy vz\n')
        for i in range(0,ip):
            fp.write(f'{i} {1} {r[i][0]} {r[i][1]} {r[i][2]} {v[i][0]} {v[i][1]} {v[i][2]}\n')

In [37]:
L = 5 # Lattice size
b = 2.0 # Size of unit cell (units of sigma)
v0 = 1.0 # Initial kinetic energy scale
N=4*L**3 # Nr of atoms
r = np.zeros((N,3))
v = np.zeros((N,3))
bvec = np.array([[0, 0, 0], [b/2, b/2, 0], [b/2, 0, b/2], [0, b/2, b/2]])
ip = 0
# Generate positions
for ix in range(0,L):
    for iy in range(0,L):
        for iz in range(0,L):
            r0 = b*np.array([ix, iy, iz]) # Unit cell base position
            for k in range(0,4):
                r[ip] = r0 + bvec[k]
                ip = ip + 1 # Add particle
            # Generate velocities
            for i in range(0,ip):
                v[i] = v0*np.random.rand(1,3);
Lx = L*b
Ly = L*b
Lz = L*b
# Output to file
writelammps('mymdinit.lammpstrj',L*b,L*b,L*b, 0, r,v)


In [25]:
# A generic Lennard Jones potential
def potential(dr):
    rr = np.dot(dr,dr)
    return -24*(2*(1/rr)**6-(1/rr)**3)*dr/rr

In [70]:
def integrator( r, v, Lx, Ly, Lz, Dump_file,t = 3.0, dt = 0.001): 
    ''' Input_file : string
        Dump_file  : string'''
    L = [Lx, Ly, Lz] 
    N = len(r) 
    n = int(np.ceil(t/dt))
    for i in range(1,n): # Loop over timesteps
        a = np.zeros((N,3)) # Store calculated accelerations
        #velocity verlet 
        for i1 in range(0, N):
            for i2 in range(i1+1, N):
                dr = r[i1] - r[i2]
                for k in range(0,3): #Periodic boundary conditions
                    if dr[k] > L[k]/2:
                        dr[k] = dr[k] - L[k]
                    elif dr[k] < -L[k]/2:
                        dr[k] = dr[k] + L[k]
                aa = potential(dr)
                a[i1] = + aa # from i2 on i1
                a[i2] = - aa # from i1 on i2
        v = v + a*dt/2;
        r = r + v*dt/2
        # Periodic boundary conditions
        for i1 in range(0, N): 
            for k in range(0,3):
                if (r[i1][k]>L[k]):
                    r[i1][k] = r[i1][k] - L[k]
                if (r[i1][k]<0):
                    r[i1][k] = r[i1][k] + L[k]
        #Other half of velocity verlet            
        for i1 in range(0, N):
            for i2 in range(i1+1, N):
                dr = r[i1] - r[i2]
                for k in range(0,3): #Periodic boundary conditions
                    if dr[k] > L[k]/2:
                        dr[k] = dr[k] - L[k]
                    elif dr[k] < -L[k]/2:
                        dr[k] = dr[k] + L[k]
                aa = potential(dr)
                a[i1] = + aa # from i2 on i1
                a[i2] = - aa # from i1 on i2
        v = v + a*dt/2;
        writelammps(Dump_file, Lx, Ly, Lz, i*dt, r, v);

In [72]:
%%time
integrator( r, v, Lx, Ly, Lz, 'mymddump.lammpstrj', t = 2, dt = 0.01)

Wall time: 6min 35s
